# ICU / intl

In [103]:
out(function () {
    yield ['Blabla', '💩'];
}, $locales);

[
  "cs_CZ" => [
    "Blabla: 💩",
  ],
  "ja_JP" => [
    "Blabla: 💩",
  ],
]

In [1]:
function canonicalize ($item) {
    if (is_array($item)) {
        return implode(', ', $item);
    }
    
    if (is_null($item)) {
        return 'null';
    }
    
    if (is_bool($item)) {
        return $item ? 'true' : 'false';
    }
    
    if (is_int($item) || is_float($item)) {
        return "{$item}";
    }
    
    if (is_string($item)) {
        return $item;
    }
    
    return "";
}

function single_out(callable $function, ...$args) {
    $out = [];
    foreach ($function(...$args) as $item) {
        if (is_null($item[0])) {
            $out[] = canonicalize($item[1]);
            continue;
        }
        
        $key = canonicalize($item[0]);
        $value = canonicalize($item[1]);
        $out[] = "{$key}: {$value}";
    }
    return $out;
}

function out (callable $function, ?array $locales = null) {
    if (is_null($locales)) {
        return single_out($function);
    }
    
    $out = [];
    foreach ($locales as $locale) {
        $out[$locale] = single_out($function, $locale);
    }
    
    return $out;
}

$locales = ['cs_CZ', 'ja_JP'];

[
  "cs_CZ",
  "ja_JP",
]

## Úvod

Jistě již Vás někdy potkala některá z následujících situací:

* Našli jste Česko v seznamu zemí zahrabané až kdesi za Zimbabwe.
* Zvolili jste špatné datum, protože týden v grafickém kalendáři nezačínal pondělím.
* Nebyli jste si jisti, která část data je den a která měsíc.
* Zkopírovali jste „2 souborů“.
* Přišel Vám e-mail nadepsaný „Byl(a) jste s našimi službami spokojen(a)?“
* Nevěděli jste, v jakém časovém pásmu jsou uvedeny časy v jízdním řádu.
* Místo popisků tlačítek jste viděli jen výpustku.
* Nechtěně jste zadali stovky místo jednotek kvůli desetinné tečce místo čárky.

Nejen to nám pomáhá vyřešit knihovna ICU a databáze CLDR.

Spousta programátorů by byla ráda, kdyby všichni na světě používali jednotný systém prakticky všeho, nejlépe i jen jediný jazyk. Některým by snad stačilo, kdyby všichni aspoň používali latinku a arabské číslice. Jenže… lidi nezměníte a software děláte pro ně.

Podpora cizích jazyků je zásadní. Nesmí se stát, že rozdíly v jazyce, znakové sadě či národním prostředí způsobí, že aplikace nebude fungovat. Dobře udělaná lokalizace a přizpůsobení národnímu prostředí, které není zásadní pro funkčnost, je dalším krokem: výrazně zlepšuje to, jak je nám používání aplikace příjemné.

### Příklady

* Nejhorší: Kvůli rozdílu v nastavení oddělovačů tisíců nepůjde odeslat formulář s částkou.
* Nepříjemné: Nadpisy článků, které nejsou latinkou, vytvoří nesrozumitelné adresy: _------_ místo _privet_.
* Kosmetické: Datum se zobrazí s jiným pořadím jednotlivých součástí.

### Co už jsem slyšel

* Sestavujte věty tak, aby nebylo potřeba měnit tvary slov.
* Nemůžeme ty ruské e-maily posílat napsané latinkou?

## IntlDateFormatter

In [2]:
out(function () {
    $dateFormat = IntlDateFormatter::LONG;
    $timeFormat = IntlDateFormatter::LONG;
    $timeZone = new DateTimeZone('Europe/Prague');
    $formatter = new IntlDateFormatter('cs_CZ', $dateFormat, $timeFormat, $timeZone);
    $dateTime = new DateTime;
    yield [null, $formatter->format($dateTime)];
}, $locales);

[
  "cs_CZ" => [
    "20. října 2018 5:50:08 SELČ",
  ],
  "ja_JP" => [
    "20. října 2018 5:50:08 SELČ",
  ],
]

## NumberFormatter

In [14]:
out(function ($locale) {
    $formatter = new NumberFormatter($locale, NumberFormatter::DECIMAL);
    yield ['default', $formatter->format(123456789.0123456)];
    $formatter->setAttribute(NumberFormatter::MAX_FRACTION_DIGITS, 4);
    yield ['MAX_FRACTION_DIGITS', $formatter->format(123456789.0123456)];
}, $locales);

[
  "cs_CZ" => [
    "default: 123 456 789,012",
    "MAX_FRACTION_DIGITS: 123 456 789,0123",
  ],
  "ja_JP" => [
    "default: 123,456,789.012",
    "MAX_FRACTION_DIGITS: 123,456,789.0123",
  ],
]

In [15]:
out(function ($locale) {
    $formatter = new NumberFormatter($locale, NumberFormatter::SPELLOUT);
    yield [null, $formatter->format(123456)];
}, $locales);

[
  "cs_CZ" => [
    "sto dvacet tři tisíc čtyři sta padesát šest",
  ],
  "ja_JP" => [
    "十二万三千四百五十六",
  ],
]

In [5]:
out(function ($locale) {
    $formatter = new NumberFormatter($locale, NumberFormatter::CURRENCY);
    yield [null, $formatter->format(1234.5678)];
}, $locales);

[
  "cs_CZ" => [
    "1 234,57 Kč",
  ],
  "ja_JP" => [
    "￥1,235",
  ],
]

## Collator

In [6]:
out(function () {
    $birds = ['chřástal', 'čížek', 'ťuhýk', 'datel', 'čáp', 'turpan', 'cetie'];

    sort($birds);
    yield ['sort', $birds];
    
    $collator = new Collator('cs_CZ');
    $collator->sort($birds);

    yield ['Collator#sort', $birds];
});

[
  "sort: cetie, chřástal, datel, turpan, čáp, čížek, ťuhýk",
  "Collator#sort: cetie, čáp, čížek, datel, chřástal, ťuhýk, turpan",
]

In [26]:
out(function () {
    $words = ['šarka', 'šąla', 'itin', 'ypač', 'suolas', 'ilgas'];
    $collator = new Collator('lt_LT');
    $collator->sort($words);
    
    yield [null, $words];
});

[
  "ilgas, ypač, itin, suolas, šąla, šarka",
]

Příklad ze života: Telefonní seznam některých starších Nokií správně řadil *ch* až za *h*. Vyhledávání však zpracovával postupně po jednotlivých písmenech. Jména obsahující *ch* tak nebylo možné vyhledat zadáním *ch* do vyhledávacího pole.

## MessageFormatter

Už jsem slyšel i výzvy: tak ty věty pište tak, aby se slova neměnila v závislosti na čísle.

> Počet jablek, která měla babka: 4.

Jako vážně?

In [7]:
out(function() {
    foreach ($counts = [5, 1.1] as $count) {
        yield ["{$count}", "Měla babka $count jablek."];
    }
});

[
  "5: Měla babka 5 jablek.",
  "1.1: Měla babka 1.1 jablek.",
]

In [8]:
out(function () {
    $pattern = 'Měla babka {count, number} {count, plural, one {jablko} '  .
                                                               'few {jablka} ' . 
                                                               'many {jablka} ' .
                                                               'other {jablek}}.';
    $formatter = new MessageFormatter('cs_CZ', $pattern);

    $counts = [1, 4, 5, 5.5, 0];
    foreach ($counts as $count) {
        yield ["{$count}", $formatter->format(['count' => $count])];
    }
});

[
  "1: Měla babka 1 jablko.",
  "4: Měla babka 4 jablka.",
  "5: Měla babka 5 jablek.",
  "5.5: Měla babka 5,5 jablka.",
  "0: Měla babka 0 jablek.",
]

In [9]:
out(function() {
    $pattern = '{gender, select, m {{subject} měl jablka.} ' .
                                'f {{subject} měla jablka.} ' .
                                'n {{subject} mělo jablka.} ' .
                                'other {{subject} si nezaslouží jablka.}}';
    $formatter = new MessageFormatter('cs_CZ', $pattern);

    $examples = [['subject' => 'Dědek', 'gender' => 'm'],
                 ['subject' => 'Babka', 'gender' => 'f'],
                 ['subject' => 'Vnouče', 'gender' => 'n']];
    foreach ($examples as $example) {
        yield [$example, $formatter->format($example)];
    }
});

[
  "Dědek, m: Dědek měl jablka.",
  "Babka, f: Babka měla jablka.",
  "Vnouče, n: Vnouče mělo jablka.",
]

## Normalizer

In [10]:
out(function () {
    $original = 'k' . 'u' . '̊' . 'n' . '̌';
    yield ['original', $original];
    yield ['original mb_strlen', mb_strlen($original)];
    yield ['original Normalizer::isNormalized', Normalizer::isNormalized($original)];

    $normalized = Normalizer::normalize($original);
    
    yield ['normalized', $normalized];
    yield ['normalized mb_strlen', mb_strlen($normalized)];
    yield ['normalized Normalizer::isNormalized', Normalizer::isNormalized($normalized)];
    
    yield ['==', $original == $normalized];
});

[
  "original: kůň",
  "original mb_strlen: 5",
  "original Normalizer::isNormalized: false",
  "normalized: kůň",
  "normalized mb_strlen: 3",
  "normalized Normalizer::isNormalized: true",
  "==: false",
]

Narazil jsem na chybu v jádře Ruby: při vytvoření souboru na souborovém systému HFS souborový systém název souboru normalizuje, když se předá nenormalizovaný. Ruby si ale ponechalo původní nenormalizovaný název, pod kterým zapsaný soubor nebylo možné nalézt.

## Transliterate

In [101]:
out(function () {
    $ids = ['uk-uk_Latn/BGN', 'ru-ru_Latn/BGN'];
    foreach ($ids as $id) {
        $transliterator = Transliterator::create($id);
        yield [$id, $transliterator->transliterate('Луганск')];
    }

    $ids = ['Upper', 'tr-Upper'];
    foreach ($ids as $id) {
        $transliterator = Transliterator::create($id);
        yield [$id, $transliterator->transliterate('binbir')];
    }
});

[
  "uk-uk_Latn/BGN: Luhansk",
  "ru-ru_Latn/BGN: Lugansk",
  "Upper: BINBIR",
  "tr-Upper: BİNBİR",
]

## IntlChar

In [11]:
out(function () {
    $chars = ['ň', '℃', 'ふ', '갨', '🍺'];
    foreach ($chars as $char) {
        yield [$char, IntlChar::charname($char)];
    }
});

[
  "ň: LATIN SMALL LETTER N WITH CARON",
  "℃: DEGREE CELSIUS",
  "ふ: HIRAGANA LETTER HU",
  "갨: HANGUL SYLLABLE GAELS",
  "🍺: BEER MUG",
]

In [12]:
out(function () {
    $chars = ['Ň', 'ň', '🍺'];
    foreach ($chars as $char) {
        yield ["{$char} islower", IntlChar::islower($char)];
        yield ["{$char} isgraph", IntlChar::isgraph($char)];
    }
});

[
  "Ň islower: false",
  "Ň isgraph: true",
  "ň islower: true",
  "ň isgraph: true",
  "🍺 islower: false",
  "🍺 isgraph: true",
]

In [13]:
out(function () {
    $char = ' ';
    $trimmed = trim($char);
    
    yield ['mb_strlen', mb_strlen($trimmed)];
    yield ['IntlChar::isWhiteSpace', IntlChar::isWhiteSpace($char)];
    yield ['IntlChar::isUWhiteSpace', IntlChar::isUWhiteSpace($char)];
});

[
  "mb_strlen: 1",
  "IntlChar::isWhiteSpace: false",
  "IntlChar::isUWhiteSpace: true",
]

# 🧜🏿‍♂️